In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores.faiss import FAISS
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import re
import os
import pprint

TOKEN_LIMIT = 32000

In [7]:
chat = AzureChatOpenAI(deployment_name="gpt4-32k", model_name="gpt-4-32k")

In [40]:
def read_text_by_path(
    path: Path,
    root_dir=Path("/Users/leung/Workspace/personal/sherlock/simple_repo/src"),
):
    with open(root_dir / path, "r") as f:
        return f.read()


def format_code_with_path(path: Path, content: str):
    return f"{path}\n{content}\n"


def extract_path_from_answer(
    answer: str,
):  # yolo hardcode
    # Get all text that is wrapped in ***
    path = re.findall(r"\*\*\*(.*?)\*\*\*", answer)
    return path


def extract_notes_from_answer(answer: str):
    # Get all text that is wrapped in ***
    notes = re.findall(r"\*\*\*(.*?)\*\*\*", answer, re.MULTILINE | re.DOTALL)
    return notes


test = """
***
main.py
The issue in main.py is in the following line:
`node_queue.put((-min_node.next.val, counter, min_node.next))`
The problem is that the value is being negated, which causes the order of the elements in the priority queue to be incorrect. To fix this issue, we need to remove the negation and change the line to:
`node_queue.put((min_node.next.val, counter, min_node.next))`
***
"""

extract_notes_from_answer(test)

['\nmain.py\nThe issue in main.py is in the following line:\n`node_queue.put((-min_node.next.val, counter, min_node.next))`\nThe problem is that the value is being negated, which causes the order of the elements in the priority queue to be incorrect. To fix this issue, we need to remove the negation and change the line to:\n`node_queue.put((min_node.next.val, counter, min_node.next))`\n']

In [9]:
TEST_ISSUE = """
## Why
Main.py do not generate correct output when I run `python main.py`


## What
Expected output:
[1, 1, 2, 3, 4, 4, 5, 6]

Got output:
[1, 4, 5, 1, 3, 4, 2, 6]
"""

TEST_DIRECTORY = """
    ./simple_repo/src/main.py
"""

In [41]:
RELATED_FILES_TEMPLATE = """
The bug report is:
```
{issue}
```

Here's a list of files in the repository:
```
{filepaths}
```

Please suggest which files we should investigate to help identify the source of the issue. If you believe that examining files would not be helpful in this case, please respond with an empty list.

For each file you suggest, please provide a brief rationale (e.g. it contains relevant code, it's connected to the affected feature, etc.) and limit your suggestion to a maximum of {token_limit} tokens.

Response in following format:

***file1.py***
Why: It contains relevant code
"""

related_files_prompt = PromptTemplate(
    input_variables=["issue", "filepaths", "token_limit"],
    template=RELATED_FILES_TEMPLATE,
)
# test
related_files_prompt.format(
    issue="test.py don't run", filepaths="./test.py", token_limit=TOKEN_LIMIT
)

"\nThe bug report is:\n```\ntest.py don't run\n```\n\nHere's a list of files in the repository:\n```\n./test.py\n```\n\nPlease suggest which files we should investigate to help identify the source of the issue. If you believe that examining files would not be helpful in this case, please respond with an empty list.\n\nFor each file you suggest, please provide a brief rationale (e.g. it contains relevant code, it's connected to the affected feature, etc.) and limit your suggestion to a maximum of 32000 tokens.\n\nResponse in following format:\n\n***file1.py***\nWhy: It contains relevant code\n"

In [11]:
def find_related_files(issue_str, filepaths_str):
    # Test with test issue
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=related_files_prompt,
    )
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
    chain = LLMChain(llm=chat, prompt=chat_prompt_template)
    ans = chain.run(issue=issue_str, filepaths=filepaths_str, token_limit=TOKEN_LIMIT)
    print(ans)
    return extract_path_from_answer(ans)


find_related_files(TEST_ISSUE, TEST_DIRECTORY)

***./simple_repo/src/main.py***
Why: It contains the main script that generates the output and where the issue is most likely to be found.


['./simple_repo/src/main.py']

In [42]:
READ_CODE_TEMPLATE = """
We received an issue report with the following description:
```
{issue}
```
We've identified the following files in the codebase that may be related to the issue:
```
{code}
```
Please read the relevant code and take some notes that will help us plan our code commits to resolve the issue. Specifically, we'd like you to note what is going wrong and suggest how we can fix the problem mentioned in issue within a maximum of {token_limit} tokens.

Format for note of each file:
***
file1.py
note for file1.py
***
***
file2.py
note for file2.py
***
"""

read_code_prompt = PromptTemplate(
    input_variables=["issue", "code", "token_limit"],
    template=READ_CODE_TEMPLATE,
)
# test
read_code_prompt.format(
    issue="test.py don't run", code="print('hello world\")", token_limit=TOKEN_LIMIT
)

'\nWe received an issue report with the following description:\n```\ntest.py don\'t run\n```\nWe\'ve identified the following files in the codebase that may be related to the issue:\n```\nprint(\'hello world")\n```\nPlease read the relevant code and take some notes that will help us plan our code commits to resolve the issue. Specifically, we\'d like you to note what is going wrong and suggest how we can fix the problem mentioned in issue within a maximum of 32000 tokens.\n\nFormat for note of each file:\n***\nfile1.py\nnote for file1.py\n***\n***\nfile2.py\nnote for file2.py\n***\n'

In [43]:
test = """***main.py***

Why: The bug report specifically mentions `main.py` as the source of the incorrect output. It is likely that the code responsible for generating the output is located in this file."""

"\n".join(
    [
        format_code_with_path(p, read_text_by_path(p))
        for p in extract_path_from_answer(test)
    ]
)

'main.py\nfrom queue import PriorityQueue\nfrom typing import List, Optional, Tuple\n\n\nclass ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\n\nclass Solution:\n    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:\n        if not lists:\n            return None\n        node_queue: PriorityQueue[Tuple[int, int, ListNode]] = PriorityQueue(len(lists))\n        # Use counter to diff same value nodes.\n        counter = 1\n        for node in lists:\n            if node:\n                node_queue.put((node.val, counter, node))\n                counter += 1\n\n        dummy_node = ListNode(0, None)\n        current = dummy_node\n\n        while not node_queue.empty():\n            _, _, min_node = node_queue.get()\n            current.next = min_node\n            current = current.next\n            if min_node.next:\n                node_queue.put((-min_node.next.val, counter, min_node.next))\n        

In [44]:
# Test with test issue


def read_code(issue_str, code_files_strs, code_file_path):
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=read_code_prompt,
    )
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
    chain = LLMChain(llm=chat, prompt=chat_prompt_template)
    concat_code = "\n".join(
        [
            format_code_with_path(p, read_text_by_path(p, code_file_path))
            for p in code_files_strs
        ]
    )
    ans = chain.run(
        issue=issue_str,
        code=concat_code,
        token_limit=TOKEN_LIMIT,
    )
    print(ans)
    return extract_notes_from_answer(ans)


read_code(
    TEST_ISSUE,
    ["main.py"],
    Path("/Users/leung/Workspace/personal/sherlock/simple_repo/src"),
)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists..


***
main.py
The issue in main.py is with the following line:
`node_queue.put((-min_node.next.val, counter, min_node.next))`
Here, the negative sign before the value leads to the wrong order of nodes in the PriorityQueue. To fix the issue, we need to remove the negative sign:
`node_queue.put((min_node.next.val, counter, min_node.next))`
***


['\nmain.py\nThe issue in main.py is with the following line:\n`node_queue.put((-min_node.next.val, counter, min_node.next))`\nHere, the negative sign before the value leads to the wrong order of nodes in the PriorityQueue. To fix the issue, we need to remove the negative sign:\n`node_queue.put((min_node.next.val, counter, min_node.next))`\n']

['\nmain.py\nThe issue in main.py is in the following line:\n`node_queue.put((-min_node.next.val, counter, min_node.next))`\nThe problem is that the value is being negated, which causes the order of the elements in the priority queue to be incorrect. To fix this issue, we need to remove the negation and change the line to:\n`node_queue.put((min_node.next.val, counter, min_node.next))`\n']

In [69]:
PLAN_PR_TEMPLATE = """
Hello, we have investigated the reported issue and would like to propose a solution. Here are the notes we gathered while examining the repository:

{notes}

The original issue can be found here:
{issue}

Please provide the following information for the pull request in a valid serialized JSON string (readable by json.loads()), and make sure to wrap the result JSON with the delimiter "***":

Pull request description should be in markdown format with the following sections:
    - Section "Why": A brief description of the issue, and why it happened
    - Section "What": A brief description of the proposed solution

Kindly ensure that the necessary files are specified for each commit. Note that folders will be created automatically, so there is no need to make separate commits for them. Please limit your response to a maximum of {token_limit} tokens.

***
{{
    "title": "Pull request title",
    "description": "Pull request description in markdown format that follows the format above",
    "commits": [
        {{
            "commit_message": "commit1",
            "commit_files": ["file_path1", "file_path2"]
        }},
        {{
            "commit_message": "commit2",
            "commit_files": ["file_path3", "file_path4"]
        }}
    ]
}}
***

"""
plan_pr_prompt = PromptTemplate(
    input_variables=["notes", "issue", "token_limit"],
    template=PLAN_PR_TEMPLATE,
)
# test
plan_pr_prompt.format(
    issue="test.py don't run", notes="print('hello world\")", token_limit=TOKEN_LIMIT
)

'\nHello, we have investigated the reported issue and would like to propose a solution. Here are the notes we gathered while examining the repository:\n\nprint(\'hello world")\n\nThe original issue can be found here:\ntest.py don\'t run\n\nPlease provide the following information for the pull request in a valid serialized JSON string (readable by json.loads()), and make sure to wrap the result JSON with the delimiter "***":\n\nPull request description should be in markdown format with the following sections:\n    - Section "Why": A brief description of the issue, and why it happened\n    - Section "What": A brief description of the proposed solution\n\nKindly ensure that the necessary files are specified for each commit. Note that folders will be created automatically, so there is no need to make separate commits for them. Please limit your response to a maximum of 32000 tokens.\n\n***\n{\n    "title": "Pull request title",\n    "description": "Pull request description in markdown form

In [73]:
import json


# Test with test issue
def extract_pr_from_answer(answer: str):
    # Get all text that is wrapped in ***
    raw = re.findall(r"\*\*\*(.*?)\*\*\*", answer, re.MULTILINE | re.DOTALL)
    # convert text to dict
    try:
        if raw[0]:
            return json.loads(raw[0])
    except IndexError:
        return json.loads(answer)  # HACK ai sometime don't respect delimiter

    return None


def plan_pr(issue_str, notes):
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=plan_pr_prompt,
    )
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
    chain = LLMChain(llm=chat, prompt=chat_prompt_template)
    ans = chain.run(
        issue=issue_str,
        notes="\n".join(notes),
        token_limit=TOKEN_LIMIT,
    )
    print(ans)
    print(extract_pr_from_answer(ans))
    return extract_pr_from_answer(ans)


plan_pr(
    TEST_ISSUE,
    [
        """
***
main.py
The issue in main.py is in the following line:
`node_queue.put((-min_node.next.val, counter, min_node.next))`
The problem is that the value is being negated, which causes the order of the elements in the priority queue to be incorrect. To fix this issue, we need to remove the negation and change the line to:
`node_queue.put((min_node.next.val, counter, min_node.next))`
***
"""
    ],
)

***
{
    "title": "Fix incorrect output in main.py",
    "description": "## Why\nMain.py did not generate the correct output when running `python main.py` due to an issue with the negation of the value in the priority queue.\n\n## What\nThe proposed solution is to remove the negation, which fixes the ordering of the elements in the priority queue and provides the expected output.",
    "commits": [
        {
            "commit_message": "Remove negation in priority queue",
            "commit_files": ["main.py"]
        }
    ]
}
***
{'title': 'Fix incorrect output in main.py', 'description': '## Why\nMain.py did not generate the correct output when running `python main.py` due to an issue with the negation of the value in the priority queue.\n\n## What\nThe proposed solution is to remove the negation, which fixes the ordering of the elements in the priority queue and provides the expected output.', 'commits': [{'commit_message': 'Remove negation in priority queue', 'commit_files': [

{'title': 'Fix incorrect output in main.py',
 'description': '## Why\nMain.py did not generate the correct output when running `python main.py` due to an issue with the negation of the value in the priority queue.\n\n## What\nThe proposed solution is to remove the negation, which fixes the ordering of the elements in the priority queue and provides the expected output.',
 'commits': [{'commit_message': 'Remove negation in priority queue',
   'commit_files': ['main.py']}]}

In [74]:
# Recursively walk through the directory and return a list of all files and their relative paths
# With the given globs
def walk_dir_for_path(dir_path: Path, globs):
    files = []
    for path in dir_path.rglob("*"):
        if path.is_file():
            if any([path.match(glob) for glob in globs]):
                files.append(path.relative_to(dir_path))
    return files


def get_code_files_path(root_dir: Path):
    return walk_dir_for_path(
        root_dir, ["*.py", "*.js", "*.ts", "*.html", "*.css", "*.go", "*.tsx", "*.jsx"]
    )


def run_chain(issue_path: Path, code_path: Path):
    issue_str = read_text_by_path(issue_path)
    code_file_list = "\n".join(
        map(lambda x: x.as_posix(), get_code_files_path(code_path))
    )
    print("### Searching for related files ###")
    related_file = find_related_files(issue_str, code_file_list)
    print("### Analysis code in related file ###")
    notes_on_code = read_code(issue_str, related_file, code_path)
    print("### Writing first draft of the pull request ###")
    suggested_pr = plan_pr(issue_str, notes_on_code)

    return {
        "related_file": related_file,
        "notes": notes_on_code,
        "suggested_pr": suggested_pr,
    }

In [75]:
test_pr = run_chain(
    issue_path=Path("/Users/leung/Workspace/personal/sherlock/simple_repo/issues/1.md"),
    code_path=Path("/Users/leung/Workspace/personal/sherlock/simple_repo/src"),
)

### Searching for related files ###
***main.py***
Why: It contains the relevant code and is the only file in the repository, so the issue must be in this file.
### Analysis code in related file ###
***
main.py
The issue is with this line of code:
`node_queue.put((-min_node.next.val, counter, min_node.next))`

The negative sign before `min_node.next.val` is causing the PriorityQueue to be sorted in descending order. To fix this issue, remove the negative sign and change the line of code to:
`node_queue.put((min_node.next.val, counter, min_node.next))`
***
### Writing first draft of the pull request ###
***
{
    "title": "Fix PriorityQueue sorting in main.py",
    "description": "## Why\n\nMain.py do not generate correct output when running `python main.py` due to incorrect sorting in the PriorityQueue.\n\n## What\n\nThis PR fixes the issue by removing the negative sign before `min_node.next.val`, ensuring that the PriorityQueue is sorted in ascending order.",
    "commits": [
        {

In [76]:
gbf_pr = run_chain(
    issue_path=Path(
        "/Users/leung/Workspace/personal/sherlock/gbf-raid-finder/issues/test.md"
    ),
    code_path=Path("/Users/leung/Workspace/personal/sherlock/gbf-raid-finder"),
)

### Searching for related files ###
***internal/clients/hub.go***
Why: It manages all the clients and handles the incoming messages.

***internal/clients/client.go***
Why: It represents each individual client and might have issues while sending messages to the frontend.

***internal/fetcher/message.go***
Why: It processes the raw messages from the Twitter API and might have issues while forwarding them.

***internal/fetcher/fetcher.go***
Why: It fetches the raw messages from the Twitter API and might not be sending them to the next processing stage.

***internal/router/router.go***
Why: It handles the routing of messages between different components and might be causing issues in message delivery.

***web/src/atoms/wsAtoms.ts***
Why: It manages the websocket connections and state in the frontend and might not be receiving messages.

***web/src/utils/messages.ts***
Why: It contains utility functions for handling messages and might have issues while processing incoming messages.

***web/

In [ ]:
test_pr

{'related_file': ['main.py'],
 'notes': ["\nmain.py\nThe problem in main.py is in the line where the next node is added to the priority queue: `node_queue.put((-min_node.next.val, counter, min_node.next))`. It's using the negative value of the node which is causing the incorrect order of the output.\n\nTo fix the problem, simply change the line to: `node_queue.put((min_node.next.val, counter, min_node.next))`\n"],
 'suggested_pr': {'title': 'Fix incorrect output order in main.py',
  'description': '### Why\n\nMain.py do not generate correct output when I run `python main.py`\n\n### What\n\nFix the issue by changing the line adding the next node to the priority queue to use the positive value of the node, ensuring the correct order of the output.',
  'commits': [{'commit_message': 'Fix incorrect output order in main.py',
    'commit_files': ['main.py']}]}}

In [77]:
PLAN_COMMIT_TEMPLATE = """
We received issue:
{issue}

And we examined the repository and wrote the following notes:
{notes}

Here is the code we may want to changes

{code_path}
{code}

Now we want you to generate a diff for this file in the following format:

***
```diff
code diff
```
***

If you think it is not required to change the code, please write `no change` in the diff.
Please make sure all changes are included in the diff. Please limit your response to a maximum of {token_limit} tokens.
"""
plan_code_prompt = PromptTemplate(
    input_variables=["issue", "notes", "code_path", "code", "token_limit"],
    template=PLAN_COMMIT_TEMPLATE,
)


def generate_diff(issue, notes, code_path):
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=plan_code_prompt,
    )
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
    chain = LLMChain(llm=chat, prompt=chat_prompt_template)
    code = read_text_by_path(code_path)
    ans = chain.run(
        issue=issue,
        notes="\n".join(notes),
        code_path=code_path.as_posix(),
        code=code,
        token_limit=TOKEN_LIMIT,
    )
    print(ans)
    return ans

In [48]:
def codegen_from_pr(issue_path, analysis, code_path):
    suggested_pr = analysis["suggested_pr"]
    issue = read_text_by_path(issue_path)
    commits = []
    notes = analysis["notes"]
    for commit in suggested_pr["commits"]:
        changes = []
        for f in commit["commit_files"]:
            change = generate_diff(
                issue=issue,
                notes=notes,
                code_path=code_path / f,
            )
            changes.append(change)
        commits.append({"message": commit["commit_message"], "changes": changes})
    print(commits)
    return commits

In [49]:
issue = TEST_ISSUE
res = codegen_from_pr(
    analysis=test_pr,
    issue_path=Path("/Users/leung/Workspace/personal/sherlock/simple_repo/issues/1.md"),
    code_path=Path("/Users/leung/Workspace/personal/sherlock/simple_repo/src"),
)
res

```diff
--- main.py
+++ main.py
@@ -30,7 +30,7 @@
             current = current.next
             if min_node.next:
-                node_queue.put((-min_node.next.val, counter, min_node.next))
+                node_queue.put((min_node.next.val, counter, min_node.next))
                 counter += 1

         return dummy_node.next
```
[{'message': 'Fix output by correcting priority queue sorting', 'changes': ['```diff\n--- main.py\n+++ main.py\n@@ -30,7 +30,7 @@\n             current = current.next\n             if min_node.next:\n-                node_queue.put((-min_node.next.val, counter, min_node.next))\n+                node_queue.put((min_node.next.val, counter, min_node.next))\n                 counter += 1\n\n         return dummy_node.next\n```']}]


[{'message': 'Fix output by correcting priority queue sorting',
  'changes': ['```diff\n--- main.py\n+++ main.py\n@@ -30,7 +30,7 @@\n             current = current.next\n             if min_node.next:\n-                node_queue.put((-min_node.next.val, counter, min_node.next))\n+                node_queue.put((min_node.next.val, counter, min_node.next))\n                 counter += 1\n\n         return dummy_node.next\n```']}]

In [55]:
SUMMARISE_TEMPLATE = """
We received issue:
{issue}

And we examined the repository and wrote the following notes:
{notes}

And we prepared the follow pr messages
{pr_desc}

And we generated the following changes:
{changes}

Now we need to write the final pull request to present our plan and changes to the user. Please write the final pull request in the following format in markdown. Please limit your response to a maximum of {token_limit} tokens

# {pr_title}

## Why

xxx

## What

xxx

### Changes

1. Commit: xxx
    ```
    diff
    ```
    Explanation: xxx

Please show changes that contains actual code changes. If there is no code changes, please skip the commit.
"""
write_pr_prompt = PromptTemplate(
    input_variables=["issue", "notes", "pr_desc", "changes", "token_limit", "pr_title"],
    template=SUMMARISE_TEMPLATE,
)


def changes_to_str(changes):
    newline = "\n"  # HACK
    changes_str = [
        f"Commit: {c['message']} \n { newline.join(c['changes'])}" for c in changes
    ]
    return "\n".join(changes_str)


def generate_final_pr(issues, analysis, commits):
    human_message_prompt = HumanMessagePromptTemplate(
        prompt=write_pr_prompt,
    )
    chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
    chain = LLMChain(llm=chat, prompt=chat_prompt_template)
    ans = chain.run(
        issue=issue,
        notes=analysis["notes"],
        pr_desc=analysis["suggested_pr"]["description"],
        pr_title=analysis["suggested_pr"]["title"],
        changes=changes_to_str(commits),
        token_limit=TOKEN_LIMIT,
    )
    print(ans)
    return ans


generate_final_pr(TEST_ISSUE, test_pr, res)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists..


# Fix output by correcting priority queue sorting

## Why

Main.py did not generate the correct output when running `python main.py` due to an issue with the sorting order in the priority queue.

## What

The proposed solution is to remove the negative sign before `min_node.next.val` in the line where nodes are put back into the priority queue. This will correctly sort the nodes in the priority queue, resulting in the expected output.

### Changes

1. Commit: Fix output by correcting priority queue sorting 
    ```diff
    --- main.py
    +++ main.py
    @@ -30,7 +30,7 @@
                 current = current.next
                 if min_node.next:
    -                node_queue.put((-min_node.next.val, counter, min_node.next))
    +                node_queue.put((min_node.next.val, counter, min_node.next))
                     counter += 1

             return dummy_node.next
    ```
    Explanation: Removed the negative sign before `min_node.next.val` to correctly sort the nodes in the

'# Fix output by correcting priority queue sorting\n\n## Why\n\nMain.py did not generate the correct output when running `python main.py` due to an issue with the sorting order in the priority queue.\n\n## What\n\nThe proposed solution is to remove the negative sign before `min_node.next.val` in the line where nodes are put back into the priority queue. This will correctly sort the nodes in the priority queue, resulting in the expected output.\n\n### Changes\n\n1. Commit: Fix output by correcting priority queue sorting \n    ```diff\n    --- main.py\n    +++ main.py\n    @@ -30,7 +30,7 @@\n                 current = current.next\n                 if min_node.next:\n    -                node_queue.put((-min_node.next.val, counter, min_node.next))\n    +                node_queue.put((min_node.next.val, counter, min_node.next))\n                     counter += 1\n\n             return dummy_node.next\n    ```\n    Explanation: Removed the negative sign before `min_node.next.val` to corre

# Demo 1


In [51]:
from IPython.display import display, Markdown

issue = Path("/Users/leung/Workspace/personal/sherlock/simple_repo/issues/1.md")
code = Path("/Users/leung/Workspace/personal/sherlock/simple_repo/src")

analysis = run_chain(issue, code)

pprint.pprint(object=analysis, indent=2)

code = codegen_from_pr(issue, analysis, code)

pprint.pprint(object=code, indent=2)


final_pr = generate_final_pr(issue, analysis, code)
display(Markdown(final_pr))

### Searching for related files ###
***main.py***
Why: It contains relevant code and is the only file in the repository
### Analysis code in related file ###
***
main.py
The issue is caused by the wrong sign in the following line:
`node_queue.put((-min_node.next.val, counter, min_node.next))`
It should be:
`node_queue.put((min_node.next.val, counter, min_node.next))`

To fix the issue, change the line as mentioned above, and the output will be correct.
***
### Writing first draft of the pull request ###
***
{
    "title": "Fix incorrect output in main.py",
    "description": "### Why\n\nMain.py do not generate correct output when running `python main.py`. The issue is caused by the wrong sign in a line of code.\n\n### What\n\nChange the line with the wrong sign to fix the issue, and the output will be correct.",
    "commits": [
        {
            "commit_message": "Fix wrong sign in main.py",
            "commit_files": ["main.py"]
        }
    ]
}
***
{'title': 'Fix incorrect out

# Fix incorrect output in main.py

## Why

Main.py does not generate the correct output when running `python main.py`. The issue is caused by the wrong sign in a line of code.

## What

Change the line with the wrong sign to fix the issue, and the output will be correct.

### Changes

1. Commit: Fix wrong sign in main.py
    ```diff
    --- main.py
    +++ main.py
    @@ -27,7 +27,7 @@
                 current.next = min_node
                 current = current.next
                 if min_node.next:
    -                node_queue.put((-min_node.next.val, counter, min_node.next))
    +                node_queue.put((min_node.next.val, counter, min_node.next))
                     counter += 1

             return dummy_node.next
    ```
    Explanation: The issue is caused by the wrong sign in the following line:
    `node_queue.put((-min_node.next.val, counter, min_node.next))`
    It should be:
    `node_queue.put((min_node.next.val, counter, min_node.next))`
    To fix the issue, change the line as mentioned above, and the output will be correct.

In [78]:
issue = Path("/Users/leung/Workspace/personal/sherlock/gbf-raid-finder/issues/test.md")
code = Path("/Users/leung/Workspace/personal/sherlock/gbf-raid-finder")

analysis = run_chain(issue, code)

pprint.pprint(object=analysis, indent=2)

code = codegen_from_pr(issue, analysis, code)

pprint.pprint(object=code, indent=2)


final_pr = generate_final_pr(issue, analysis, code)
display(Markdown(final_pr))

### Searching for related files ###
***internal/clients/hub.go***
Why: It manages the connection between clients and the app, which might not be sending messages to the frontend.

***internal/clients/client.go***
Why: It handles individual client connections and could be the cause of the issue if messages are not being sent properly.

***internal/fetcher/message.go***
Why: It processes the messages from Twitter API and could be causing an issue if messages are not being passed onto the frontend.

***internal/fetcher/fetcher.go***
Why: It fetches messages from Twitter API, and there might be an issue with how they are being handled.

***internal/router/router.go***
Why: It handles routing within the application, and there could be an issue with the websocket route.

***web/src/atoms/wsAtoms.ts***
Why: It manages the websocket state for the frontend, and there might be an issue with how messages are being processed.

***web/src/utils/messages.ts***
Why: It contains utility functions to h

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15

There is no specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions. Please follow the possible solutions listed above to identify and resolve the issue.
```diff
no change
```
```diff
no change
```


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 4 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.chat_models.openai.ChatOpen

no change


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


no change
[{'message': 'Fix registration and unregistration of clients in hub.go', 'changes': ['```diff\nno change\n```']}, {'message': 'Fix read and write pumps in client.go', 'changes': ['There is no specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions. Please follow the possible solutions listed above to identify and resolve the issue.']}, {'message': 'Fix tweet message transformation in message.go', 'changes': ['```diff\nno change\n```']}, {'message': 'Fix Twitter API connection and tweet stream handling in fetcher.go', 'changes': ['```diff\nno change\n```']}, {'message': 'Fix websocket atoms management in wsAtoms.ts', 'changes': ['no change']}, {'message': 'Fix tweet board rendering and communication with websocket atoms in index.tsx', 'changes': ['no change']}]
[ { 'changes': ['```diff\nno change\n```'],
    'message': 'Fix registration and unregistration of clients in hub.go'},
  { 'changes

# Fix WebSocket issue for Twitter messages

## Why

No twitter message pushed to frontend. The issue might be related to the handling of websocket connections, message broadcasting, and Twitter API connection.

## What

This pull request proposes a solution to address the issue by checking and fixing the handling of websocket connections, message broadcasting, and Twitter API connection in the internal/clients, internal/fetcher, and web/src components.

### Changes

1. Fix registration and unregistration of clients in hub.go

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions. 

2. Fix read and write pumps in client.go

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions.

3. Fix tweet message transformation in message.go

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions.

4. Fix Twitter API connection and tweet stream handling in fetcher.go

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions.

5. Fix websocket atoms management in wsAtoms.ts

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions.

6. Fix tweet board rendering and communication with websocket atoms in index.tsx

    No specific code change to suggest at this moment, as the issue requires further investigation and debugging in the mentioned files and functions.

Please follow the possible solutions listed above to identify and resolve the issue.

# Test 3


In [79]:
issue = Path("/Users/leung/Workspace/personal/sherlock/subee/issues/test.md")
code = Path("/Users/leung/Workspace/personal/sherlock/subee")

analysis = run_chain(issue, code)

pprint.pprint(object=analysis, indent=2)

code = codegen_from_pr(issue, analysis, code)

pprint.pprint(object=code, indent=2)


final_pr = generate_final_pr(issue, analysis, code)
display(Markdown(final_pr))

### Searching for related files ###
***options.go***
Why: It contains the code for handling options, which is where the WithConcurrency option should be added.

***engine.go***
Why: It contains the main engine code, which is where the concurrency logic should be implemented.

***engine_test.go***
Why: It contains tests for the engine, which is where new tests for the WithConcurrency option should be added.
### Analysis code in related file ###
***
options.go
- Add a new option function WithConcurrency to set the concurrency level (number of goroutines)
***
```go
// WithConcurrency returns an Option that sets the concurrency level for consuming messages.
func WithConcurrency(concurrency int) Option {
	return func(c *Config) {
		if concurrency > 0 {
			c.Concurrency = concurrency
		}
	}
}
```
- Update the Config struct to include a Concurrency field
***
```go
type Config struct {
	// ...
	Concurrency            int
	// ...
}
```
- Update the newDefaultConfig function to set the default c

# Add WithConcurrency option to control the number of goroutines

## Why

There was a need to control the number of goroutines that consume messages concurrently. The default behavior was to consume messages using a single goroutine.

## What

This pull request adds a new `WithConcurrency` option to set the desired concurrency level (number of goroutines). By using this option, one can control the number of goroutines that consume messages concurrently. The default concurrency level is set to 1.

### Changes

1. Commit: Add WithConcurrency option to options.go
    ```diff
    diff --git a/options.go b/options.go
    index 1234567..abcdefg 100644
    --- a/options.go
    +++ b/options.go
    @@ -14,6 +14,18 @@ func WithConsumerInterceptors(interceptors ...ConsumerInterceptor) Option {
     }
    }

    +// WithConcurrency returns an Option that sets the number of goroutines that will consume messages concurrently.
    +func WithConcurrency(concurrency int) Option {
    +	return func(c *Config) {
    +		if concurrency > 0 {
    +			c.Concurrency = concurrency
    +		} else {
    +			c.Concurrency = 1
    +		}
    +	}
    +}
    +
    // WithBatchConsumerInterceptors returns an Option that sets the BatchConsumerInterceptor implementations(s).
    // Interceptors are called in order of addition.
    // e.g) interceptor1, interceptor2, interceptor3 => interceptor1 => interceptor2 => interceptor3 => BatchConsumer.Consume
    ```

2. Commit: Update engine.go to use the concurrency setting
    ```diff
    diff --git a/engine.go b/engine.go
    index 6c42d6b..b6c8fbd 100644
    --- a/engine.go
    +++ b/engine.go
    @@ -21,6 +21,7 @@ func NewBatch(subscriber Subscriber, consumer BatchConsumer, opts ...Option) *En
     cfg := newDefaultConfig()
     cfg.BatchConsumer = bConsumer
     cfg.Consumer = consumer
    +	cfg.Concurrency = 1
     cfg.apply(opts)

     e := &Engine{
    @@ -33,12 +34,20 @@ func newEngine(subscriber Subscriber, bConsumer BatchConsumer, consumer Consume
    // Start starts Subscriber and Consumer process.
    func (e *Engine) Start(ctx context.Context) error {
     e.Logger.Print("Start Pub/Sub worker")
    -	defer e.Logger.Print("Finish Pub/Sub worker")
    +	defer e.Logger.Print("Finish all Pub/Sub worker")

     ctx = setLogger(ctx, e.Logger)

    -	return errors.WithStack(newProcess(e).Start(ctx))
    +	var wg sync.WaitGroup
    +	for i := 0; i < e.Config.Concurrency; i++ {
    +		wg.Add(1)
    +		go func() {
    +			defer wg.Done()
    +			err := errors.WithStack(newProcess(e).Start(ctx))
    +			if err != nil {
    +				e.Logger.Printf("Error in the process: %v", err)
    +			}
    +		}()
    +	}
    +	wg.Wait()
    +	return nil
    }
    ```